In [1]:
import galois
import numpy as np
from py_ecc.bn128 import G1, G2, multiply, curve_order, neg, pairing, Z1, Z2
import py_ecc.bn128
import random

In [2]:
GF = galois.GF(curve_order)

In [3]:
def evaluate_polynomial_galois(coefs, x, GF):
    result = GF(0)
    for coef in coefs:
        result = result * x + GF(coef)
    return int(result)

def interpolate_lagrange(col, GF, num_eq):
    xs = GF(np.array([i for i in range(1, num_eq+1)]))
    poly = galois.lagrange_poly(xs, col)
    poly_return = np.pad(poly.coeffs, (max(num_eq - len(poly.coeffs), 0), 0), 'constant') #To have vectors of the same size
    return poly_return

def trusted_setup(U,V,W):
 
    m=len(U)-1
    n=len(U[0])

    #========================== Pour trouver t(x)  ==========================
    values = [i for i in range(1, len(U[0])+1)]
    poly_t = galois.Poly([1], field=GF)
    for val in values:
        poly_t *= galois.Poly([1, -val], field=GF)
    t=[]
    for elem in poly_t.coeffs:
        t.append(int(elem))
    #=========================================================================
    
    x=GF(random.randint(1,curve_order-1))
    print("x = ",int(x))

    t_x=evaluate_polynomial_galois(t, x, GF)

    alpha =GF(random.randint(1,curve_order-1))
    beta =GF(random.randint(1,curve_order-1))

    print("alpha = ",int(alpha))
    print("beta = ",int(beta))
    
    U_polys=[]
    V_polys=[]
    W_polys=[]

    for elem in U:
        U_polys.append(galois.Poly((elem),field=GF))
    for elem in V:
        V_polys.append(galois.Poly((elem),field=GF))
    for elem in W:
        W_polys.append(galois.Poly((elem),field=GF))
    polys=[]
    for i in range(0, m+1):
        u_i=U_polys[i](x)
        v_i=V_polys[i](x)
        w_i=W_polys[i](x)
        polys.append((int(beta)*u_i+int(alpha)*v_i+w_i))

    x_power_i=[]
    for i in range(0, n):
        x_power_i.append(x**i)

    x_power_i_t_x=[]
    for i in range(0,n-2+1):#[0,n-1]
        x_power_i_t_x.append((x**i)*t_x)

    power_c=[]
    for i in range(0,n):
        power_c.append(x_power_i[i]*t_x)
    
    G1_x_power_i = [multiply(G1,int(elem)) for elem in x_power_i]
    G1_x_power_i_t_x = [multiply(G1,int(elem)) for elem in x_power_i_t_x]
    G2_x_power_i = [multiply(G2,int(elem)) for elem in x_power_i]
    G1_alpha= multiply(G1,int(alpha))
    G2_beta= multiply(G2,int(beta))
    G1_poly=[multiply(G1,int(elem)) for elem in polys]

    return G1_x_power_i, G1_x_power_i_t_x, G2_x_power_i, G1_alpha, G2_beta, G1_poly

def prover(U,V,W,param,a):
    print("\n============= Prover =============")

    r=GF(random.randint(1,curve_order-1))
    s=GF(random.randint(1,curve_order-1))

    m=len(U)-1
    n=len(U[0]) 

    G1_x_power_i, G1_x_power_i_t_x, G2_x_power_i, G1_alpha, G2_beta, G1_poly = param

    #=================================Pour trouver t(x)  ==========================
    values = [i for i in range(1, len(U[0])+1)] # Car on a 6 colonnes
    poly_t = galois.Poly([1], field=GF)  # Start with the polynomial '1' in the given field
    for val in values:
        poly_t *= galois.Poly([1, -val], field=GF)
    t=[]
    for elem in poly_t.coeffs:
        t.append(int(elem))
    #=================================Pour trouver h(x)  ==========================
    Uw = galois.Poly(np.matmul(np.transpose(U), a))
    Vw = galois.Poly(np.matmul(np.transpose(V), a))
    Ww = galois.Poly(np.matmul(np.transpose(W), a))
    t_poly=galois.Poly(t,field=GF)

    h = (Uw * Vw - Ww) // t_poly
    h_rem = (Uw * Vw - Ww) % t_poly

    assert h_rem == 0, "h(x) is not a multiple of t(x)"

    h_t_x=None
    for i in range(0, len(h.coeffs)):
        h_t_x=py_ecc.bn128.add(h_t_x,multiply(G1_x_power_i_t_x[i],int(h.coeffs[::-1][i])))

    #==============================================================================

    Ui_x = []

    for i in range(0, m+1):
        temp=None
        for j in range(0, len(U[i])):
            temp2=multiply(G1_x_power_i[j],int(U[i][::-1][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        Ui_x.append(temp)
        
    A_1=None
    for i in range(0, m+1):
        temp=multiply(Ui_x[i],int(a[i]))
        A_1=py_ecc.bn128.add(A_1,temp)
    A_1 = py_ecc.bn128.add(A_1, G1_alpha)
    print("A_1=",A_1)

    V_i_x_2 = []
    for i in range(0, m+1):
        temp=None
        for j in range(0, len(V[i])):
            temp2=multiply(G2_x_power_i[j],int(V[i][::-1][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        V_i_x_2.append(temp)

    B_2=None
    for i in range(0, m+1):
        temp=multiply(V_i_x_2[i],int(a[i]))
        B_2=py_ecc.bn128.add(B_2,temp)
    B_2 = py_ecc.bn128.add(B_2, G2_beta)
    print("B_2=",B_2)

    C_1=None
    for i in range(0, m+1):
        temp=multiply(G1_poly[i],int(a[i]))
        C_1=py_ecc.bn128.add(C_1,temp)
    C_1 = py_ecc.bn128.add(C_1, h_t_x)

    print("C_1=",C_1)
    return (A_1, B_2, C_1)

def verifier(pi,a,param):
    print("\n============= Verifier =============")

    A_1, B_2, C_1 = pi

    G1_x_power_i, G1_x_power_i_t_x, G2_x_power_i, G1_alpha, G2_beta, G1_poly = param

    pairing1 = pairing(B_2, A_1)
    pairing2= pairing(G2_beta, G1_alpha)
    pairing3 = pairing(G2, C_1)

    return pairing1==pairing2*pairing3

In [4]:
import numpy as np

O = np.array([
[0,0,0,1,0,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1],
[0,1,0,0,0,0,0,0]
])
L = np.array([
[ 0,0,1,0,0,0,0,0],
[ 0,0,1,0,0,0,0,0],
[ 3,0,0,0,0,0,0,0],
[ 5,0,0,0,0,0,0,0],
[10,0,0,0,0,0,0,0],
[ 3,0,0,0,0,1,1,1]
])
R = np.array([
[0,0,1,0,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,1,0,0,0,0,0],
[1,0,0,0,0,0,0,0]
])

w=[1, 553, 5, 25, 125, 375, 125, 50]
result = O.dot(w) == np.multiply(L.dot(w),R.dot(w))

assert result.all(), "Le produit ne fonctionne pas"
print(" → Vecteur choisi", w)

 → Vecteur choisi [1, 553, 5, 25, 125, 375, 125, 50]


In [5]:
num_eq = len(L)

p=curve_order

L_galois = GF(np.array(L) % p)
R_galois = GF(np.array(R) % p)
O_galois = GF(np.array(O) % p)

U = []
V = []
W = []

for i in range(len(w)):
    U.append(interpolate_lagrange(L_galois[:, i], GF, num_eq))
    V.append(interpolate_lagrange(R_galois[:, i], GF, num_eq))
    W.append(interpolate_lagrange(O_galois[:, i], GF, num_eq))

print("Matrix U:")
for line in U:
    for elem in line:
        print(elem, end=" ")
    print()

print("\nMatrix V:")
for line in V:
    for elem in line:
        print(elem, end=" ")
    print()

print("\nMatrix W:")
for line in W:
    for elem in line:
        print(elem, end=" ")
    print()
    
w=GF(w)



Matrix U:
9302503220531691969454722441734341912633054870176814596071736779294718610637 6384070837619788606488535009033371900826606283454676683578642887751277477892 2736030358979909402780800718157159386068545550052004292962275523321976061929 4560050598299849004634667863595265643447575916753340488270459205536626769988 20793830728247311461134085457994411334120946180395232626513293977247018070746 42 
0 0 0 0 0 0 
12403337627375589292606296588979122550177406493569086128095649039059624814183 13680151794899547013904003590785796930342727750260021464811377616609880309760 12768141675239577212977070018066743801653212566909353367157285775502554955781 8208091076939728208342402154471478158205636650156012878886826569965928185842 18605006441063383938909444883468683825266109740353629192143473558589437221295 21888242871839275222246405745257275088548364400416034343698204186575808495608 
0 0 0 0 0 0 
0 0 0 0 0 0 
8572895124803716128713175583559099409681442723496280117948463306408858327450 2736030358979909

In [6]:
U=GF(U)
V=GF(V)
W=GF(W)

param= trusted_setup(U, V, W)
G1_x_power_i, G1_x_power_i_t_x, G2_x_power_i, G1_alpha, G2_beta, G1_poly = param
print("\n============= Trusted Setup =============")
print("G1_x_power_i=",G1_x_power_i)
print("G1_x_power_i_t_x=",G1_x_power_i_t_x)
print("G2_x_power_i=",G2_x_power_i)
print("G1_alpha=",G1_alpha)
print("G2_beta=",G2_beta)
print("G1_poly=",G1_poly)


pi = prover(U, V, W, param, w)
isTrue = verifier(pi, w, param)
print(isTrue)

x =  2817708336798945090966936456850407956579976228379687715496732105334346984022
alpha =  6154777329126321447277902892499319251174327708041231244536883150498727870855
beta =  16347227563234109016088675402624100850527027897964622011825505295207166471839

============= Prover =============
A_1= (645512954972625242961716719833822006225753834118013129822050793902246726140, 12253473832199413035278672275640322204934867612045617165953264651932983327922)
B_2= ((9292717671992072821284298184732667731046589781466024412189607654935219745787, 13741582064480655469007844329167958910036373790375831223628671724340159561048), (20055470788211806189469040340943136647679004105945591555514408828811620256402, 12033646219042632633671753664722203430027811083536870160623957991185303315254))
C_1= (19930577126301452464309699410643330709141006345385141804750957186613351791553, 7773619077087962582919027877597054821552146478047105437649419123909692061720)


TypeError: verifier() missing 1 required positional argument: 'l'